## ボートレース 公式サイトからScrapeを行いpickleファイルに保存

In [1]:
from datetime import datetime
from datetime import timedelta
from http.client import RemoteDisconnected
from bs4 import BeautifulSoup
import urllib.request
import time
import pandas as pd
import re

### web pageから情報を取ってきてpandas dfに格納するモジュール

In [26]:
def scrape_racelist(soup, rno, jcd, hd):
    """
    racelistのページに書かれている情報をクロール
    :return:
    """
    table = soup.find(class_="contentsFrame1_inner").find_all(class_="table1")[1]
    rows = table.find_all("tbody", {"class": "is-fs12"})
    
    race_result_dict_list = []
    
    for i, row in enumerate(rows, 1):
        race_result_dict = {"date": "-".join([hd[0:4], hd[5:7], hd[8:10]]),
                            "venue": jcd, "raceNumber": rno[:-1]
                           }
        # 枠
        race_result_dict["枠"] = i
        # racer id
        race_result_dict["racer_id"] = row.find(class_="is-fs11").text.split("\n")[1][-6:-2]
        race_result_dict["racer_class"] = row.find(class_="is-fs11").text.split("\n")[2][-2:]

        # 選手名。最後の[1:-1]は改行を削除するため
        racer_name = row.find(class_="is-fs18 is-fBold").text[1:-1]
        
        # race_result_listの要素としてクロールした結果のリストを追加
        race_result_dict["racer_name"] = racer_name

        # racer data
        racer_column_3 = row.find_all("td", {"class": "is-lineH2"})[0].text.split("\n")
        race_result_dict["num_false_start"] = racer_column_3[1][-3:-1]
        race_result_dict["num_late_start"] = racer_column_3[2][-3:-1]

        # crawl motor data
        motor_column = row.find_all("td", {"class": " is-lineH2"})[0].text.split("\n")
        race_result_dict["motorNo"] = motor_column[1][-4:-1]
        race_result_dict["モーター2連率"] = motor_column[2][-7:-1]
        race_result_dict["モーター3連率"] = motor_column[3][-7:-1]

        # crawl boat data
        boat_column = row.find_all("td", {"class": " is-lineH2"})[1].text.split("\n")
        race_result_dict["boatNo"] = boat_column[1][-4:-1]
        race_result_dict["ボート2連率"] = boat_column[2][-7:-1]
        race_result_dict["ボート3連率"] = boat_column[3][-7:-1]
        
        race_result_dict_list.append(race_result_dict)
        
    # dictをdfに変換
    race_result_df = pd.DataFrame.from_dict(race_result_dict_list)

    time.sleep(0.1)

    return race_result_df

In [3]:
def scrape_beforeinfo(soup, rno, jcd, hd):
    """
    exhibitionの情報など、直前情報ページに書かれている情報をクロール
    :param soup:
    :param rno:
    :param jcd:
    :param hd:
    :return:

    # TODO: プロペラ
    # TODO: 部品交換
    # TODO: 前走成績
    # TODO: 調整重量 (adjustment weight) (kg)
    # TODO: 風向き

    """
    race_result_dict_list = []
    
    table = soup.find(class_="contentsFrame1_inner").find_all(class_="table1")[1]
    rows = table.find_all("tbody", {"class": "is-fs12"})
    
    table2 = soup.find(class_="contentsFrame1_inner").find_all(class_="table1")[2]
    rows2 = table2.find_all("tr")


    for i, (row, row2) in enumerate(zip(rows, rows2[2:]), 1):
        
        race_result_dict = {"date": "-".join([hd[0:4], hd[5:7], hd[8:10]]),
                        "venue": jcd,
                        "raceNumber": rno[:-1]
                        }
        # 枠
        race_result_dict["枠"] = i
        
        # 水面気象情報
        table3 = soup.find(class_="contentsFrame1_inner").find(class_="weather1")
        weather_data = (table3.find_all(class_="weather1_bodyUnitLabelData"))
        weather_string = table3.find_all(class_="weather1_bodyUnitLabelTitle")

        race_result_dict["temperature"] = weather_data[0].text[:-1]
        race_result_dict["weather"] = weather_string[1].text
        race_result_dict["wind_speed"] = weather_data[1].text[:-1]
        race_result_dict["water_temperature"] = weather_data[2].text[:-1]
        race_result_dict["wave_height"] = weather_data[3].text[:-2]

        # racer weight (kg)
        # 書いていないことがあり、その場合エラーになる
        race_result_dict["weight"] = row.find("td", {"rowspan": "2"}).text[:-2]

        # 展示タイム
        race_result_dict["exhibitionTime"] = row.find_all("td", {"rowspan": "4"})[3].text

        # チルト角度
        race_result_dict["tilt"] = row.find_all("td", {"rowspan": "4"})[4].text
        
        # 展示競争での進入コース
        race_result_dict["exhibition_cource"] = row2.find_all("span")[0].text
        # 展示start time (ST, flyng, late)
        ex_st_ = row2.find_all("span")[3].text
        if len(ex_st_) == 3:
            race_result_dict["exhibition_ST"] = ex_st_
            race_result_dict["flying"] = 0
            race_result_dict["late"] = 0

        elif len(ex_st_) == 4:
            race_result_dict["exhibition_ST"] = ex_st_[1:]
            if ex_st_[0] == "F":
                race_result_dict["flying"] = 1
                race_result_dict["late"] = 0
            # elif ex_st_[0] == "L":
            #     race_result_dict["late_{0}".format(i)] = 1
            else:
                raise Exception("{0}号艇ex_stが予定外（{1}）".format(i, ex_st_))
        elif len(ex_st_) == 1:
            if ex_st_[0] == "L":
                race_result_dict["exhibition_ST"] = None
                race_result_dict["late"] = 1
            else:
                raise Exception("{0}号艇ex_stが予定外（{1}）".format(i, ex_st_))

        else:
            raise Exception("{0}号艇ex_stが予定外（{1}）".format(i, ex_st_))
        
        # 最初に定義したリストに辞書型のデータを追加
        race_result_dict_list.append(race_result_dict)

    # dictを入れたlistをdfに変換
    beforeinfo_df = pd.DataFrame.from_dict(race_result_dict_list)

    time.sleep(0.1)

    return beforeinfo_df

In [4]:
def scrape_raceresult(soup, rno, jcd, hd):
    
    race_result_dict_list = []
    
    table = soup.find(class_="contentsFrame1_inner").find_all(class_="table1")[1]
    rows = table.find_all("tbody")
    
    for row in rows:
        race_result_dict = {"date": "-".join([hd[0:4], hd[5:7], hd[8:10]]),
                            "venue": jcd,
                            "raceNumber": rno[:-1]
                            }
        race_result_dict["着順"] = row.find_all("td")[0].text
        # 枠番はintegerにしておかないとconcatした時に別の行として扱われてしまう
        race_result_dict["枠"] = int(row.find_all("td")[1].text)
        race_result_dict["タイム"] = row.find_all("td")[3].text
        
        # 最初に定義したリストに辞書型のデータを追加
        race_result_dict_list.append(race_result_dict)
    
    # dictを入れたlistをdfに変換
    raceresult_df = pd.DataFrame.from_dict(race_result_dict_list)

    time.sleep(0.1)

    return raceresult_df

### そのほかcrawl, scrapeに必要なモジュール

In [5]:
def make_url(crawl_key, rno, jcd, hd):
    """
    :param crawl_key: 何をcrawleするか。選択肢は、"odds3t"（オッズ）, "racelist"(出走表）,
    "beforeinfo" (直前情報）もしくは"raceresult" (レース結果)
    :param rno: レース番号。8Rなど、1-12の数字 + R をstrで
    :param jcd: 会場名。"桐　生"、"びわこ"など
    :param hd: holding day (レース開催日)、2019/03/28などyyyy/mm/ddの形で入力（strで）
    :return dds_url: 公式サイト最終オッズが書かれているページのurl. これを使ってcrawlする
    """
    jcd_dict =  {"桐　生": "01", "戸　田": "02", "江戸川": "03", "平和島": "04", "多摩川": "05", "浜名湖": "06", "蒲　郡": "07", "常　滑": "08",
                "　津　": "09", "三　国": "10", "びわこ": "11", "住之江": "12", "尼　崎": "13", "鳴　門": "14", "丸　亀": "15", "児　島": "16",
                "宮　島": "17", "徳　山": "18", "下　関": "19", "若　松": "20", "芦　屋": "21", "福　岡": "22", "唐　津": "23", "大　村": "24"
                }
    rno = rno[:-1]
    hd = hd[0:4] + hd[5:7] + hd[8:10]

    odds_url = "http://boatrace.jp/owpc/pc/race/" + crawl_key + "?rno=" + rno + "&jcd=" + jcd_dict[jcd] + "&hd=" + hd

    return odds_url


def html_parser(site_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",
    }

    try:
        request = urllib.request.Request(url=site_url, headers=headers)
        response = urllib.request.urlopen(request)

        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'lxml')

    # データベース作成の際、remotedisconnectedになった場合,そのレースをパス
    except RemoteDisconnected:
        print("remote disconnected error !")
        return None

    except ConnectionResetError:
        print("Connection Reset error !")
        return None

    return soup

def get_extractor(crawl_key):
    
    """
    クロール先に応じたcrawlerを用意
    
    """
    
    extractor_dict = {"racelist": scrape_racelist,
                      "beforeinfo": scrape_beforeinfo,
                      "raceresult": scrape_raceresult,
                      }
    
    return extractor_dict[crawl_key]

### 実行部分

In [29]:
hd_list = ["2021/03/05", "2021/03/06", "2021/03/7"]

crawl_key_list = ["racelist", "beforeinfo", "raceresult"]
jcd_list =  ["桐　生", "戸　田", "江戸川", "平和島", "多摩川", "浜名湖", "蒲　郡", "常　滑",
                "　津　", "三　国", "びわこ", "住之江", "尼　崎", "鳴　門", "丸　亀", "児　島",
                "宮　島", "徳　山", "下　関", "若　松", "芦　屋", "福　岡", "唐　津", "大　村"
            ]

for hd in hd_list:

    # 1日単位でデータを集めてファイルに保存する
    today_race_df_list = []

    for jcd in jcd_list:
        for i in range(1, 13):
            rno = str(i) + "R"

            # その日レースがない場所は飛ばすためのtry-except         
            try:
                # 色々なkeyに対してクロールして特定のレースの情報がまとまったdfを作る
                race_info_df_list = []

                for crawl_key in crawl_key_list:
                    raceResult_url = make_url(crawl_key, rno, jcd, hd)

                    # パース
                    soup = html_parser(raceResult_url)

                    # extractorの指定
                    the_extractor = get_extractor(crawl_key)

                    # 対象サイトをcrawl
                    race_information_df = the_extractor(soup, rno, jcd, hd)
                    race_information_df = race_information_df.set_index(["date", "venue", "raceNumber", "枠"])

                    race_info_df_list.append(race_information_df)

                this_race_df =pd.concat(race_info_df_list, axis=1, join_axes=[race_info_df_list[0].index])
                # 今回のレースのデータを本日のデータを集めたリストに格納
                today_race_df_list.append(this_race_df)

            except IndexError:
                print(hd + " " + jcd + rno +"データなし")
                pass

    # 本日のレースデータを集めたリストをdfに変換    
    today_race_df = pd.concat(today_race_df_list, axis = 0)

    # pickleファイルで保存
    today_race_df.to_pickle('./crawledData/{0}.pkl'.format("".join(hd.split("/"))))

2021/03/05 戸　田1Rデータなし
2021/03/05 戸　田2Rデータなし
2021/03/05 戸　田3Rデータなし
2021/03/05 戸　田4Rデータなし
2021/03/05 戸　田5Rデータなし
2021/03/05 戸　田6Rデータなし
2021/03/05 戸　田7Rデータなし
2021/03/05 戸　田8Rデータなし
2021/03/05 戸　田9Rデータなし
2021/03/05 戸　田10Rデータなし
2021/03/05 戸　田11Rデータなし
2021/03/05 戸　田12Rデータなし
2021/03/05 江戸川11Rデータなし
2021/03/05 多摩川1Rデータなし
2021/03/05 多摩川2Rデータなし
2021/03/05 多摩川3Rデータなし
2021/03/05 多摩川4Rデータなし
2021/03/05 多摩川5Rデータなし
2021/03/05 多摩川6Rデータなし
2021/03/05 多摩川7Rデータなし
2021/03/05 多摩川8Rデータなし
2021/03/05 多摩川9Rデータなし
2021/03/05 多摩川10Rデータなし
2021/03/05 多摩川11Rデータなし
2021/03/05 多摩川12Rデータなし
2021/03/05 蒲　郡1Rデータなし
2021/03/05 蒲　郡2Rデータなし
2021/03/05 蒲　郡3Rデータなし
2021/03/05 蒲　郡4Rデータなし
2021/03/05 蒲　郡5Rデータなし
2021/03/05 蒲　郡6Rデータなし
2021/03/05 蒲　郡7Rデータなし
2021/03/05 蒲　郡8Rデータなし
2021/03/05 蒲　郡9Rデータなし
2021/03/05 蒲　郡10Rデータなし
2021/03/05 蒲　郡11Rデータなし
2021/03/05 蒲　郡12Rデータなし
2021/03/05 　津　1Rデータなし
2021/03/05 　津　2Rデータなし
2021/03/05 　津　3Rデータなし
2021/03/05 　津　4Rデータなし
2021/03/05 　津　5Rデータなし
2021/03/05 　津　6Rデータなし
2021/03/05 　津　7Rデータなし
2021/03/05 　津　8Rデータなし


2021/03/7 　津　8Rデータなし
2021/03/7 　津　9Rデータなし
2021/03/7 　津　10Rデータなし
2021/03/7 　津　11Rデータなし
2021/03/7 　津　12Rデータなし
2021/03/7 三　国1Rデータなし
2021/03/7 三　国2Rデータなし
2021/03/7 三　国3Rデータなし
2021/03/7 三　国4Rデータなし
2021/03/7 三　国5Rデータなし
2021/03/7 三　国6Rデータなし
2021/03/7 三　国7Rデータなし
2021/03/7 三　国8Rデータなし
2021/03/7 三　国9Rデータなし
2021/03/7 三　国10Rデータなし
2021/03/7 三　国11Rデータなし
2021/03/7 三　国12Rデータなし
2021/03/7 びわこ1Rデータなし
2021/03/7 びわこ2Rデータなし
2021/03/7 びわこ3Rデータなし
2021/03/7 びわこ4Rデータなし
2021/03/7 びわこ5Rデータなし
2021/03/7 びわこ6Rデータなし
2021/03/7 びわこ7Rデータなし
2021/03/7 びわこ8Rデータなし
2021/03/7 びわこ9Rデータなし
2021/03/7 びわこ10Rデータなし
2021/03/7 びわこ11Rデータなし
2021/03/7 びわこ12Rデータなし
2021/03/7 住之江1Rデータなし
2021/03/7 住之江2Rデータなし
2021/03/7 住之江3Rデータなし
2021/03/7 住之江4Rデータなし
2021/03/7 住之江5Rデータなし
2021/03/7 住之江6Rデータなし
2021/03/7 住之江7Rデータなし
2021/03/7 住之江8Rデータなし
2021/03/7 住之江9Rデータなし
2021/03/7 住之江10Rデータなし
2021/03/7 住之江11Rデータなし
2021/03/7 住之江12Rデータなし
2021/03/7 尼　崎1Rデータなし
2021/03/7 尼　崎2Rデータなし
2021/03/7 尼　崎3Rデータなし
2021/03/7 尼　崎4Rデータなし
2021/03/7 尼　崎5Rデータなし
2021/03/7 尼　崎6Rデータなし
2

ValueError: No objects to concatenate

In [9]:
# ファイル内容確認用
pd.read_pickle('./crawledData/{0}.pkl'.format("".join(hd.split("/"))))

boatNo motorNo num_false_start num_late_start  \
date       venue raceNumber 枠                                                 
2021-03-08 戸　田   1          1     66      18              F0             L0   
                            2     35      30              F0             L0   
                            3     21      43              F0             L0   
                            4     28      37              F0             L0   
                            5     19      22              F0             L0   
                            6     37      20              F0             L0   
                 2          1     44      28              F0             L0   
                            2     41      24              F0             L0   
                            3     75       4              F0             L0   
                            4     56      27              F0             L0   
                            5     61      53              F0             L0   
                            6     70      64              F0             L0   
                 3          1     20       3              F0             L0   
                            2     32      63              F0             L0   
                            3     74      13              F0             L0   
                            4     11      19              F0             L0   
                            5     52      32              F1             L0   
                            6     26      49              F0             L0   
                 4          1     40      11              F0             L0   
                            2     34       5              F0             L0   
                            3     42      57              F0             L0   
                            4     27      25              F0             L0   
                            5     25       1              F0             L0   
                            6     62      52              F0             L0   
                 5          1     15      36              F0             L0   
                            2     43       8              F0             L0   
                            3     48      58              F0             L0   
                            4     66      18              F0             L0   
                            5     12       9              F0             L0   
                            6     68      31              F0             L0   
...                              ...     ...             ...            ...   
           大　村   8          1     44      75              F0             L0   
                            2     76      18              F0             L0   
                            3     31      71              F0             L0   
                            4     60      35              F0             L0   
                            5     13      37              F0             L0   
                            6     38      47              F0             L0   
                 9          1     62      54              F0             L0   
                            2     73      52              F0             L0   
                            3     59      69              F0             L0   
                            4     64      27              F0             L0   
                            5     19      70              F0             L0   
                            6     22      76              F0             L0   
                 10         1     63      72              F0             L0   
                            2     21      13              F0             L0   
                            3     46      33              F0             L0   
                            4     14      65              F0             L0   
                            5     28      19              F0             L0   
                            6     65      31              F0             L0   
 

In [ ]:
# crawlerの動きを確認する用

crawl_key = "racelist"
jcd =  "戸　田"
hd = "2021/03/09"
rno = "1R"

raceResult_url = make_url(crawl_key, rno, jcd, hd)

# パース
soup = html_parser(raceResult_url)

# extractorの指定
the_extractor = get_extractor(crawl_key)

# 対象サイトをcrawl
race_information_df = the_extractor(soup, rno, jcd, hd)
race_information_df = race_information_df.set_index(["date", "venue", "raceNumber", "枠"])
race_information_df